# **CH02.1. RNN-base Encoder-decoder**

#### **`Paper Info`** : Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
#### $ \hspace{1.75cm} - $ Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre et al. (EMNLP/2014)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|-------------|
| Research topic | RNN Encoder–Decoder로 phrase/sentence-level conditional modeling을 수행하여 SMT의 phrase table scoring을 개선 |
| Core idea | Encoder가 variable-length source를 fixed-length vector $ c $로 압축하고 Decoder가 $ p(y \mid x) $를 autoregressive하게 <br> 모델링하며 새로운 gated hidden unit으로 안정적 학습과 장단기 의존성 포착 | 
| Key findings | $ \cdot{} $ Log-linear SMT에 RNN score를 feature로 추가 시 BLEU 상승 <br> $ \cdot{} $ 빈도 의존 통계 대신 linguistic regularity를 반영하는 representation 학습 <br> $ \cdot{} $ Long/rare phrase에서 특히 이득 |
| Contributions | $ \cdot{} $ RNN Encoder–Decoder 아키텍처 제안 <br> $ \cdot{} $ Reset/Update gates를 갖는 간결한 unit 제안(초기 GRU) <br> $ \cdot{} $ SMT phrase pair rescoring으로 상보적 성능 향상 실증 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ Phrase-based SMT는 주로 빈도 기반 phrase translation probability와 language model을 결합하여 log-linear로 디코딩하지만 rare/long phrase에서 일반화가 약함
##### $ \hspace{0.15cm} \cdot{} $ Feedforward NN 기반 접근은 fixed-size 입력/출력 제약으로 최대 길이를 작게 두어야 하고 word order 반영이 제한적임
##### $ \hspace{0.15cm} \cdot{} $ Word2Vec은 distributional similarity에 강점이 있으나 conditional generation과 sequence-to-sequence mapping에 직접적이지 않음

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ Variable-length 시퀀스를 보존된 order로 인코딩하고 조건부 분포 $ p(y \mid{} x) $를 직접 학습하여 SMT에서 의미·통사적 규칙을 반영하는 phrase scoring 제공
##### $ \hspace{0.15cm} \cdot{} $ 기존 빈도 기반 통계의 한계를 보완하여 rare/long phrase에서도 일관된 품질 확보

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ RNN Encoder–Decoder를 학습하여 source phrase $ x $를 고정 길이 표현 $ c $로 요약하고 decoder로 target phrase $ y $의 조건부분포를 모델링하여 log-linear SMT에 추가 feature로 사용
##### $ \hspace{0.15cm} \cdot{} $ Reset/Update gate를 갖는 간결한 gated unit을 채택하여 장기 정보 보존과 선택적 갱신을 동시에 달성

<b></b>

> ## **방법론(Method)**

#### **(1) RNN Language Modeling** :
##### $ \hspace{0.15cm} \cdot{} $ 입력 시퀀스 $ x = (x_{1}, \ldots, x_{T}) $에 대해 은닉 상태는 $ h_{t} = f(h_{t-1}, x_{t}) $
##### $ \hspace{0.15cm} \cdot{} $ 시점별 조건부분포는 $ p(x_{t} \mid x_{<t}) = \text{softmax}\big(W_{\text{out}} h_{t}\big) $
##### $ \hspace{0.15cm} \Rightarrow{} p(x) = \prod_{t=1}^{T} p(x_{t} \mid x_{<t}) \;\; \text{ where } \, h_{t} \in{} \mathbb{R}^{H}, \; W_{\text{out}} \in{} \mathbb{R}^{V \times H} $
#### **(2) Encoder–Decoder** :
##### $ \hspace{0.15cm} \cdot{} $ Encoder는 $ h^{\text{enc}}_{t} = f^{\text{enc}}(h^{\text{enc}}_{t-1}, x_{t}) $로 순회 후 요약벡터 $ c = \psi{}(h^{\text{enc}}_{T}) $
##### $ \hspace{0.15cm} \cdot{} $ Decoder는 $ h^{\text{dec}}_{t} = f^{\text{dec}}(h^{\text{dec}}_{t-1}, y_{t-1}, c) $
##### $ \hspace{0.15cm} \cdot{} $ 다음 심볼 조건분포는 $ p(y_{t} \mid y_{<t}, c) = \text{softmax}\big(g(h^{\text{dec}}_{t}, y_{t-1}, c)\big) $
##### $ \hspace{0.15cm} \Rightarrow{} p(y \mid x) = \prod_{t=1}^{T'} p(y_{t} \mid y_{<t}, c) \;\; \text{ where } \, c \in{} \mathbb{R}^{C}, \; g: \mathbb{R}^{H} \to \mathbb{R}^{V} $
#### **(3) 학습목표(Objective)** :
##### $ \hspace{0.15cm} \cdot{} $ 병렬 phrase 쌍 데이터 $ \mathcal{D} = \{(x^{(n)}, y^{(n)})\}_{n=1}^{N} $
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \max_{\theta{}} \; \sum_{n=1}^{N} \log{} p_{\theta{}}\big(y^{(n)} \mid x^{(n)}\big) \;\; \text{ where } \, \theta{} : \text{encoder/decoder parameters} $
#### **(4) Gated hidden unit** :
##### $ \hspace{0.15cm} \cdot{} $ Reset gate $ r_{t} = \sigma{}\big(W_{r} x_{t} + U_{r} h_{t-1}\big) $
##### $ \hspace{0.15cm} \cdot{} $ Update gate $ z_{t} = \sigma{}\big(W_{z} x_{t} + U_{z} h_{t-1}\big) $
##### $ \hspace{0.15cm} \cdot{} $ 후보 상태 $ \tilde{h}_{t} = \phi{}\big(W x_{t} + U(r_{t} \odot h_{t-1})\big) $
##### $ \hspace{0.15cm} \Rightarrow{} h_{t} = z_{t} \odot h_{t-1} + (1 - z_{t}) \odot \tilde{h}_{t} \;\; \text{ where } \, \sigma{} : \text{logistic}, \; \phi{} : \tanh{}, \; \odot : \text{Hadamard} $
##### **(`PLUS`) 게이트 해석** :
##### $ \hspace{0.15cm} \cdot{} $ $ z_{t} \to 1 $이면 항등 경로가 열려 장기 정보가 보존되고 $ z_{t} \to 0 $이면 새로운 정보로 갱신
##### $ \hspace{0.15cm} \cdot{} $ $ r_{t} \to 0 $이면 과거 의존을 억제하여 단기 패턴에 집중
#### **(5) SMT 통합(Log-linear model)** :
##### $ \hspace{0.15cm} \cdot{} $ SMT 디코딩은 $ \displaystyle{} \arg\max_{y} \sum_{k} w_{k} f_{k}(x,y) $
##### $ \hspace{0.15cm} \Rightarrow{} \text{여기에 } f_{\text{RNN}}(x,y) = \log{} p_{\theta{}}(y \mid x) \text{ 를 추가하여 가중치 } w_{k} \text{ 는 개발셋 BLEU로 튜닝 } \;\; \text{ where } \, f_{k} : \text{기존 TM/LM/penalty 등} $
##### **(`PLUS`) Frequency handling** :
##### $ \hspace{0.15cm} \cdot{} $ 학습 샘플링 시 phrase pair 빈도를 무시하여 RNN이 빈도 랭킹이 아닌 linguistic regularity를 학습하도록 유도
##### **(`PLUS`) Decoder 출력층** :
##### $ \hspace{0.15cm} \cdot{} $ Maxout + softmax를 사용하여 표현력과 안정성을 확보

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|-------------|
| Task | WMT14 En→Fr |
| Parallel words | $\approx{} 850M$ |
| LM raw words | $\approx{} 2B$ |
| Data selection | LM $418M$ words used from $2B$ |
| RNN training words | $348M$ from parallel subset |
| Dev set | newstest2012/2013 |
| Test set | newstest2014 |
| Vocab | $15k$ En, $15k$ Fr, OOV→[UNK] |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|-------------|
| Encoder/Decoder units | $1000$ gated units each |
| Embedding | rank-$100$ (effective dim $100$) |
| Decoder output | Maxout-$500$ (pool=2) + softmax |
| Optimizer | Adadelta($\rho{}=0.95,\epsilon{}=1e{-6}$) + SGD |
| Batch size | $64$ phrase pairs |
| Init | $ \mathcal{N}(0,0.01) $, recurrent via SVD left-singular vectors |
| Train time | $\approx{} 3$ days (GPU) |
| CSLM baseline | 7-gram, emb $512$, ReLU layers, val ppl $45.80$ |
| Integration | Log-linear feature add (CSLM+RNN complementary) |

#### **(2) 정량 분석 요약** :
##### $ \hspace{0.15cm} \cdot{} $ Baseline SMT 대비 RNN feature 추가로 dev/test BLEU 모두 상승
##### $ \hspace{0.15cm} \cdot{} $ CSLM + RNN 조합이 최상 성능을 보여 상보적임을 확인
##### $ \hspace{0.15cm} \cdot{} $ Word penalty를 추가하면 dev에서 미세 이득이나 test에선 제한적

#### **(3) 정성 분석 요약** :
##### $ \hspace{0.15cm} \cdot{} $ Long/rare source phrase에서 RNN score는 의미·통사 규칙을 더 잘 반영하여 자연스러운 target 후보를 상위에 배치
##### $ \hspace{0.15cm} \cdot{} $ 통계 모델은 빈도 강점, RNN은 regularity 강점으로 서로 보완적

##### **(`PLUS`) Representation 시각화** :
##### $ \hspace{0.15cm} \cdot{} $ T-SNE로 본 단어/문구 임베딩에서 의미·통사적 유사성에 따른 군집 형성 확인

<b></b>

> ## **결론(Conclusion)**

#### **(1)** RNN Encoder–Decoder는 phrase-level $ p(y \mid x) $ 모델링으로 SMT의 빈도 편향을 보완
#### **(2)** Reset/Update gate를 갖는 간결한 unit으로 장단기 의존성 학습과 안정적 최적화를 달성
#### **(3)** Log-linear SMT에 RNN score를 feature로 추가하면 CSLM과 상보적으로 BLEU가 상승
#### **(4)** Long/rare phrase에서 linguistic regularity를 포착하는 representation 학습의 이점을 정량·정성으로 확인
#### **(5)** 향후 attention 도입 및 phrase table 대체 가능성으로 확장 여지가 큼